Endprojekt GIS Analyse - Location Analysis for a new Stadium in Graz

In [74]:
# Data manipulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Geospatial
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon
import osmnx as ox

# Raster processing
import os
import pyproj
# optinal code fix with PROJ_LIB if pyproj doesnt work properly
os.environ["PROJ_LIB"] = pyproj.datadir.get_data_dir()
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import CRS
from rasterio.features import shapes
from shapely.geometry import shape


Data Preparation
===============================================================================     

Getting district boundaries of Graz (District boundaries (Overpass Turbo (n.d.). Overpass API web interface. https://overpass-turbo.eu/ (Accessed December 2, 2025))) <br>
and <br>
population statistic (Stadt Graz (2025). Zahlen + Fakten: Bevölkerung, Bezirke, Wirtschaft, Geografie. https://www.graz.at/cms/beitrag/10034466/7772565/Zahlen_Fakten_Bevoelkerung_Bezirke_Wirtschaft.html (Accessed December 2, 2025))

In [75]:
#loading GeoJson Districts of Graz
gdf_districts = gpd.read_file("data/district_graz.geojson")
gdf_districts = gdf_districts[["name", "geometry"]].copy() #extract the columns needed

#loading csv with pop data of graz 
df_popgraz = pd.read_csv("data/Bevoelkerung Graz.csv", encoding="latin1", sep=";")

#merge the two datasets
gdf_districts_popGraz = gdf_districts.merge(df_popgraz, left_on="name", right_on="Bezirk")

Getting hard exklusion layers from OpenStreetMap via OSMnx, set the CRS to 32633 (UTM zone 33N) and buffer some of them:<br>
- water bodies buffer (10m)<br>
- parks/green areas <br>
- transport infrastructure <br>
- buildings (buffer 250m for hospitals, kingardens, cemeteries)

In [76]:
PLACE_NAME:str = "Graz, Austria"
#TARGET_CRS = "EPSG:32633"  # UTM zone 33N

In [77]:
tags_water = {
    "natural": ["water"],
    "waterway": ["river", "stream", "canal", "ditch"],
    "landuse": ["reservoir"],
    "water": ["lake", "river", "pond", "basin"]
}
gdf_water = ox.features_from_place(
    PLACE_NAME,
    tags=tags_water
)
gdf_water = gdf_water[["geometry"]].copy()
gdf_water["category"] = "water"

# Reproject
gdf_water = gdf_water.to_crs(epsg=32633)

# 20 meter buffer
gdf_water_Buffer20m = gdf_water.copy()
gdf_water_Buffer20m["geometry"] = gdf_water_Buffer20m.buffer(20)
gdf_water_Buffer20m.head()

geometry category
element  id                                                                 
relation 1306807  POLYGON ((532493.308 5213424.458, 532492.762 5...    water
         2325656  POLYGON ((536884.355 5207621.79, 536885.105 52...    water
         3403202  POLYGON ((536189.617 5210881.951, 536188.455 5...    water
         3562966  POLYGON ((533575.587 5209754.372, 533574.904 5...    water
         3587716  POLYGON ((535239.727 5206817.913, 535241.479 5...    water

In [78]:
#Excklusion Parks
tags_parks = {
    "leisure": [
        "park", "garden", "playground", "recreation_ground"
    ],
    "boundary": [
        "protected_area"
    ]
}
gdf_parks = ox.features_from_place(
    PLACE_NAME,
    tags=tags_parks
)

gdf_parks = gdf_parks[["geometry"]].copy()
gdf_parks["category"] = "green areas"

# Reproject
gdf_parks = gdf_parks.to_crs(epsg=32633)

# 20 meter buffer
gdf_parks_Buffer20m = gdf_parks.copy()
gdf_parks_Buffer20m["geometry"] = gdf_parks_Buffer20m.buffer(20)
gdf_parks_Buffer20m.head(5)

geometry  \
element id                                                              
node    859421763   POLYGON ((529258.583 5215245.914, 529258.487 5...   
        940994980   POLYGON ((532495.494 5215082.215, 532495.398 5...   
        1053837597  POLYGON ((534682.168 5213142.265, 534682.072 5...   
        1265834876  POLYGON ((532026.172 5209443.055, 532026.076 5...   
        1412653685  POLYGON ((532010.365 5216204.807, 532010.269 5...   

                       category  
element id                       
node    859421763   green areas  
        940994980   green areas  
        1053837597  green areas  
        1265834876  green areas  
        1412653685  green areas

In [79]:
#Exclusion Transport Areas
tags_transport = {
    "highway": [
        "motorway", "trunk", "primary", "secondary", "tertiary",
        "motorway_link", "trunk_link", "primary_link", "secondary_link"
    ],
    "railway": [
        "rail", "tram", "light_rail", "subway"
    ]
}

gdf_transport = ox.features_from_place(
    PLACE_NAME,
    tags=tags_transport
)

gdf_transport = gdf_transport[["geometry"]].copy()
gdf_transport["category"] = "transport infrastructure"

# Reproject
gdf_transport = gdf_transport.to_crs(epsg=32633)

# 20 meter buffer
gdf_transport_Buffer20m = gdf_transport.copy()
gdf_transport_Buffer20m["geometry"] = gdf_transport_Buffer20m.buffer(20)
gdf_transport_Buffer20m.head(20)

geometry  \
element id                                                           
way     3987860  POLYGON ((533449.684 5211136.395, 533465.333 5...   
        3991673  POLYGON ((533280.286 5213193.422, 533280.399 5...   
        3991674  POLYGON ((534340.711 5211868.999, 534341.198 5...   
        3992913  POLYGON ((533593.03 5212405.682, 533593 521240...   
        3992916  POLYGON ((536352.536 5209722.242, 536355.837 5...   
        3992920  POLYGON ((536223.209 5208898.865, 536215.025 5...   
        3992921  POLYGON ((537098.264 5209773.987, 537098.543 5...   
        3993614  POLYGON ((534692.734 5212269.011, 534695.215 5...   
        3993615  POLYGON ((535012.603 5212456.471, 535015.55 52...   
        3995445  POLYGON ((535250.201 5210298.075, 535251.198 5...   
        3997179  POLYGON ((533882.629 5212763.121, 533880.227 5...   
        3997181  POLYGON ((533526.762 5212817.099, 533526.869 5...   
        3997182  POLYGON ((533796.086 5212962.12, 533771.174 52...   
        3997183  POLYGON ((533059.821 5212644.213, 533060.055 5...   
        3997185  POLYGON ((532983.79 5212614.523, 532981.831 52...   
        3997187  POLYGON ((532404.634 5212619.653, 532402.702 5...   
        3997188  POLYGON ((530730.677 5209707.406, 530732.635 5...   
        3997189  POLYGON ((531384.022 5209762.696, 531382.568 5...   
        3997190  POLYGON ((533096.522 5208533.592, 533094.993 5...   
        3997191  POLYGON ((531379.256 5208939.667, 531380.633 5...   

                                 category  
element id                                 
way     3987860  transport infrastructure  
        3991673  transport infrastructure  
        3991674  transport infrastructure  
        3992913  transport infrastructure  
        3992916  transport infrastructure  
        3992920  transport infrastructure  
        3992921  transport infrastructure  
        3993614  transport infrastructure  
        3993615  transport infrastructure  
        3995445  transport infrastructure  
        3997179  transport infrastructure  
        3997181  transport infrastructure  
        3997182  transport infrastructure  
        3997183  transport infrastructure  
        3997185  transport infrastructure  
        3997187  transport infrastructure  
        3997188  transport infrastructure  
        3997189  transport infrastructure  
        3997190  transport infrastructure  
        3997191  transport infrastructure

In [80]:
#Exclusion Buildings
tags_buildings_hard = {
    "building": True
}

gdf_buildings = ox.features_from_place(
    PLACE_NAME,
    tags=tags_buildings_hard
)

gdf_buildings = gdf_buildings[["geometry"]].copy()
gdf_buildings["category"] = "buildings_hard"

# Reproject
gdf_buildings = gdf_buildings.to_crs(epsg=32633)
gdf_buildings_Buffer30m = gdf_buildings.copy()
gdf_buildings_Buffer30m["geometry"] = gdf_buildings_Buffer30m.buffer(30)
gdf_buildings_Buffer30m.head(5)

geometry  \
element id                                                              
node    278296607   POLYGON ((532743.99 5212987.937, 532743.846 52...   
        1083590320  POLYGON ((531854.409 5213308.019, 531854.264 5...   
        1188213884  POLYGON ((533232.322 5212779.72, 533232.177 52...   
        1446305820  POLYGON ((531810.911 5214585.035, 531810.767 5...   
        1485789259  POLYGON ((532044.902 5213725.416, 532044.757 5...   

                          category  
element id                          
node    278296607   buildings_hard  
        1083590320  buildings_hard  
        1188213884  buildings_hard  
        1446305820  buildings_hard  
        1485789259  buildings_hard

In [81]:
#Cemeteries with 250m Buffer
tags_cemetery = {
    "landuse": ["cemetery"],
    "amenity": ["grave_yard"]
}

gdf_cemetery = ox.features_from_place(
    PLACE_NAME,
    tags=tags_cemetery
)

gdf_cemetery = gdf_cemetery[["geometry"]].copy()
gdf_cemetery["category"] = "cemetery"

gdf_cemetery = gdf_cemetery.to_crs(epsg=32633)
gdf_cemetery.head(5)

geometry  \
element  id                                                              
node     1447016305                      POINT (535889.182 5209509.97)   
relation 1252339     POLYGON ((531967.661 5210631.508, 531988.86 52...   
         1252342     POLYGON ((532066.233 5210275.077, 532053.994 5...   
         2694625     MULTIPOLYGON (((531233.48 5218089.831, 531255....   
         17946151    POLYGON ((534965.319 5212317.125, 534995.58 52...   

                     category  
element  id                    
node     1447016305  cemetery  
relation 1252339     cemetery  
         1252342     cemetery  
         2694625     cemetery  
         17946151    cemetery

Merge to one geodataframe and dissolve to one geometry

In [82]:
gdf_exclusion_areas = pd.concat([
    gdf_water_Buffer20m,
    gdf_parks_Buffer20m,
    gdf_transport_Buffer20m,
    gdf_buildings_Buffer30m,
    gdf_cemetery,
], ignore_index=True)

In [83]:
gdf_exclusion_areas.head(5)

,geometry,category
0,"POLYGON ((532493.308 5213424.458, 532492.762 5...",water
1,"POLYGON ((536884.355 5207621.79, 536885.105 52...",water
2,"POLYGON ((536189.617 5210881.951, 536188.455 5...",water
3,"POLYGON ((533575.587 5209754.372, 533574.904 5...",water
4,"POLYGON ((535239.727 5206817.913, 535241.479 5...",water


In [84]:
gdf_exclusion_areas.geometry.geom_type.value_counts()


Polygon         70641
MultiPolygon        7
Point               1
Name: count, dtype: int64

In [85]:
gdf_exclusion_areas = gdf_exclusion_areas[
    gdf_exclusion_areas.geometry.geom_type != "Point"
]
gdf_exclusion_areas = gdf_exclusion_areas.explode(
    ignore_index=True
)
gdf_exclusion_areas.geometry.geom_type.value_counts()

Polygon    70668
Name: count, dtype: int64

Reprojecting DEM from WGS84 to UTM33N and calculate slope 

In [86]:
dem_in = "data/DEM30_Graz.tif"          # original DEM
dem_utm = "data/DEM30_Graz_utm33.tif"   # output DEM (UTM33N)

with rasterio.open(dem_in) as src:
    src_crs = src.crs if src.crs is not None else CRS.from_epsg(4326)
    dst_crs = CRS.from_epsg(32633)  # UTM Zone 33N

    transform, width, height = calculate_default_transform(
        src_crs, dst_crs, src.width, src.height, *src.bounds
    )

    profile = src.profile.copy()
    profile.update({
        "crs": dst_crs,
        "transform": transform,
        "width": width,
        "height": height,
        "nodata": src.nodata if src.nodata is not None else -9999
    })

    with rasterio.open(dem_utm, "w", **profile) as dst:
        dst_array = np.full((height, width), profile["nodata"], dtype=profile["dtype"])
        dst.write(dst_array, 1)

        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=src.transform,
            src_crs=src_crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear,
            src_nodata=src.nodata,
            dst_nodata=profile["nodata"]
        )

In [87]:
dem_path = "data/DEM30_Graz_utm33.tif"
slope_path = "data/dem_slope.tif"

with rasterio.open(dem_path) as src:
    dem = src.read(1, masked=True).astype("float64")
    profile = src.profile.copy()
    transform = src.transform

xres = transform.a
yres = abs(transform.e)

dzdy, dzdx = np.gradient(dem.filled(np.nan), yres, xres)

slope_rad = np.arctan(np.sqrt(dzdx**2 + dzdy**2))
slope_deg = np.degrees(slope_rad)

nodata_out = -9999.0
slope_out = np.where(np.isnan(slope_deg), nodata_out, slope_deg).astype(np.float32)

profile.update(dtype=rasterio.float32, count=1, nodata=nodata_out)

with rasterio.open(slope_path, "w", **profile) as dst:
    dst.write(slope_out, 1)


Polygonize slope raster for areas with slope greater than threshold (e.g. 5 degrees)

In [88]:
slope_path = "data/dem_slope.tif"
out_path   = "data/slope_filtered.shp"

threshold   = 5.0     # Grad
min_area_m2 = 500.0   # Mindestfläche

with rasterio.open(slope_path) as src:
    slope = src.read(1)
    transform = src.transform
    crs = src.crs
    nodata = src.nodata

mask = slope > threshold
if nodata is not None:
    mask = mask & (slope != nodata)
mask = mask & np.isfinite(slope)

value_raster = mask.astype(np.uint8)

geoms = []
for geom, value in shapes(value_raster, mask=mask, transform=transform):
    if value == 1:
        geoms.append(shape(geom))

gdf_slope = gpd.GeoDataFrame(geometry=geoms, crs=crs)

gdf_slope["diss"] = 1
gdf_slope = gdf_slope.dissolve(by="diss")
gdf_slope = gdf_slope.explode(index_parts=False).reset_index(drop=True)

gdf_slope["area_m2"] = gdf_slope.area
gdf_slope = gdf_slope[gdf_slope["area_m2"] >= min_area_m2]

gdf_slope["slope_gt"] = threshold
gdf_slope = gdf_slope[["slope_gt", "area_m2", "geometry"]]
gdf_slope.to_file(out_path, driver="ESRI Shapefile")

print(f"{len(gdf_slope)} Polygone gespeichert in {out_path}")

4307 Polygone gespeichert in data/slope_filtered.shp


c:\Users\cleam\Documents\UNI\WS2025\Gat2\venv-gat2\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 166892829.33209983 of field area_m2 of feature 3466 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(


In [89]:
graz = gdf_districts_popGraz.copy()
steep = gpd.read_file("data/slope_filtered.shp")

if graz.crs != steep.crs:
    graz = graz.to_crs(steep.crs)

#fix potential geometry issues
graz["geometry"] = graz.geometry.buffer(0)
steep["geometry"] = steep.geometry.buffer(0)

#union all areas 
steep_union = steep.geometry.union_all()

#differnce
result = graz.copy()
result["geometry"] = result.geometry.difference(steep_union)

#remove empty geometries
result = result[result.geometry.notna() & ~result.is_empty]

result.to_file("data/graz_suitable_slope.shp", driver="ESRI Shapefile")



C:\Users\cleam\AppData\Local\Temp\ipykernel_17628\4066166847.py:21: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  result.to_file("data/graz_suitable_slope.shp", driver="ESRI Shapefile")
c:\Users\cleam\Documents\UNI\WS2025\Gat2\venv-gat2\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Größe in km2' to 'Größe in'
  ogr_write(
c:\Users\cleam\Documents\UNI\WS2025\Gat2\venv-gat2\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Einwohner pro km2' to 'Einwohne_1'
  ogr_write(


In [90]:
import geopandas as gpd

suitable = gpd.read_file("data/graz_suitable_slope.shp")
excluded = gdf_exclusion_areas.copy()

if suitable.crs != excluded.crs:
    excluded = excluded.to_crs(suitable.crs)

suitable["geometry"] = suitable.geometry.buffer(0)
excluded["geometry"] = excluded.geometry.buffer(0)

excluded_union = excluded.geometry.union_all()

suitable_area = suitable.copy()
suitable_area["geometry"] = suitable_area.geometry.difference(excluded_union)

suitable_area = suitable_area[suitable_area.geometry.notna() & ~suitable_area.is_empty].copy()

suitable_area = suitable_area.explode(index_parts=False).reset_index(drop=True)
suitable_area = suitable_area[suitable_area.geometry.notna() & ~suitable_area.is_empty].copy()

suitable_area["area_m2"] = suitable_area.area
suitable_area = suitable_area[suitable_area["area_m2"] >= 130000][["area_m2", "geometry"]]

suitable_area.to_file("data/graz_final_suitable_areas.shp", driver="ESRI Shapefile")
print("Fertig:", len(suitable_area), "Polygon(e)")

Fertig: 5 Polygon(e)


C:\Users\cleam\AppData\Local\Temp\ipykernel_17628\3640416386.py:17: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  suitable_area = suitable_area[suitable_area.geometry.notna() & ~suitable_area.is_empty].copy()


In [91]:
print("Graz CRS:", graz.crs)
print("Steep CRS:", steep.crs)
slope_graz = gpd.read_file("data/graz_suitable_slope.shp")
print("Slope CRS:", slope_graz.crs)

Graz CRS: EPSG:32633
Steep CRS: EPSG:32633
Slope CRS: EPSG:32633


# Weighting and Scoring

medium weight criteria when not near by kindergarten, hospital, cemetery and churches <br>
high weight criteria when near by transport infrastructure <br>

In [92]:
#filter kindergartens
tags_kindergarten = {
    "amenity": ["kindergarten"]
}

gdf_kindergarten = ox.features_from_place(
    PLACE_NAME,
    tags=tags_kindergarten
)

gdf_kindergarten = gdf_kindergarten[["geometry"]].copy()
gdf_kindergarten["category"] = "kindergarten"

gdf_kindergarten = gdf_kindergarten.to_crs(epsg=32633)
gdf_kindergarten.head(5)


geometry      category
element id                                                      
node    270508839   POINT (530445.941 5213616.972)  kindergarten
        629028254    POINT (535001.653 5212012.05)  kindergarten
        629029213   POINT (535437.684 5215756.501)  kindergarten
        855502493    POINT (532981.04 5214572.576)  kindergarten
        1244386379   POINT (531537.76 5214648.173)  kindergarten

In [93]:
#filter hospitals
tags_hospital = {
    "amenity": ["hospital"]
}

gdf_hospital = ox.features_from_place(
    PLACE_NAME,
    tags=tags_hospital
)

gdf_hospital = gdf_hospital[["geometry"]].copy()
gdf_hospital["category"] = "hospital"

gdf_hospital = gdf_hospital.to_crs(epsg=32633)
gdf_hospital.head(5)

geometry  \
element id                                                              
node    2238807934                      POINT (530040.636 5211023.45)   
way     4236777     POLYGON ((535590.696 5215182.609, 535589.492 5...   
        83810712    POLYGON ((530371.341 5214034.341, 530367.276 5...   
        83813652    POLYGON ((531765.665 5209187.656, 531790.914 5...   
        84202665    POLYGON ((530371.341 5214034.341, 530317.416 5...   

                    category  
element id                    
node    2238807934  hospital  
way     4236777     hospital  
        83810712    hospital  
        83813652    hospital  
        84202665    hospital

In [94]:
#filter churches

In [95]:
#filter cemeteries 
tags_cemetery = {
    "amenity": ["cemetery"]
}

gdf_cemetery = ox.features_from_place(
    PLACE_NAME,
    tags=tags_cemetery
)

gdf_cemetery = gdf_cemetery[["geometry"]].copy()
gdf_cemetery["category"] = "cemetery"

gdf_cemetery = gdf_cemetery.to_crs(epsg=32633)
gdf_cemetery.head(5)

InsufficientResponseError: No matching features. Check query location, tags, and log.

In [ ]:
# Public transport stops in Graz
tags_stops = {
    "highway": ["bus_stop"],
    "public_transport": ["platform", "stop_position"],
    "railway": ["tram_stop"]
}

gdf_stops = ox.features_from_place(
    PLACE_NAME,
    tags=tags_stops
)

# Nur Geometrie behalten
gdf_stops = gdf_stops[["geometry"]].copy()

# Kategorie setzen
gdf_stops["category"] = "public_transport_stop"

In [ ]:
#residental areas